# Assignment 6 (Speeding up with cython)
This noteboook provides implementations (namely python, cython and numpy) of the trapz function as described in the assignment

Author: Kaushik G Iyer (EE23B135)

## Instructions to run:
Just like run the cells one by one. All of the cells have a description of what is being done just above it : )

## The trapz function:
It provides a method to perform the closed integral of a function in some range.

It does so by approximating the area under the curve to be made of n trapezoids.

Mathematically it is given by:
$$
\int_{a}^{b} f(x) \,dx = \frac{h}{2} * [f(a) + 2 * \{\sum_{n=1}^{n-1} f(a + ih)\} + f(b)]
$$
- Where $h = \frac{b - a}{n}$

## Imports
Please install the required libraries (cython and numpy)

In [16]:
%load_ext Cython
import numpy as np
import math

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## Python Implementation

In [17]:
from typing import Callable


def py_trapz(f: Callable[[float], float], a: float, b: float, n: int) -> float:
    """
    Performs the definite integration of the function f between a and b
    NOTE: Expects a function that is integrable in the range provided!
    """   
    assert n > 0, "Cannot integerate function with less than 1 trapezoid!"
    
    if (b < a): # Handle the case when the limits are reversed
        return -py_trapz(f, b, a, n)

    step = (b - a) / n
    if (step == 0): # Handle the case when the limits are the same
        return .0

    y_sum = (f(a) + f(b)) / 2 # Notice that the coefficients of the endpoints is step/2
    x = a + step

    for _ in range(n - 1): # Go through all the required x in (a, b)
        # NOTE: We add the complexity of another int to save accuracy
        # If we did while x < b we might get extra points due to floating point errors 
        y_sum += f(x)
        x += step

    return y_sum * step



## Cython Implementation

In [18]:
%%cython --annotate

cimport cython

@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.cdivision(True)
@cython.profile(False)
cpdef double cy_trapz(f, double a, double b, int n):    
    """
    Performs the definite integration of the function f between a and b
    NOTE: Expects a function that is integrable in the range provided!
    """
    
    assert n > 0, "Cannot integerate function with less than 1 trapezoid!"

    if (b < a): # Handle the case when the limits are reversed
        return -cy_trapz(f, b, a, n)

    cdef double step = (b - a) / n
    if (step == 0): # Handle the case when the limits are the same
        return .0

    cdef double y_sum = (f(a) + f(b)) / 2 # Notice that the coefficients of the endpoints is step/2
    cdef double x = a + step

    for _ in range(n - 1): # Go through all the required x in (a, b)
        # NOTE: We add the complexity of another int to save accuracy
        # If we did while x < b we might get extra points due to floating point errors
        y_sum += f(x)
        x += step

    return y_sum * step

In [19]:
%%cython --annotate
cimport cython

from cpython.pycapsule cimport PyCapsule_GetPointer,  PyCapsule_GetName
ctypedef double(*function_type)(double) 

@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.cdivision(True)
@cython.profile(False)
cdef double cy_trapz_plus_helper(function_type f, double a, double b, int n):
    """
    Performs the definite integration of the function f between a and b
    NOTE: Expects a function that is integrable in the range provided!
    """
    
    assert n > 0, "Cannot integerate function with less than 1 trapezoid!"

    if (b < a): # Handle the case when the limits are reversed
        return -cy_trapz_plus_helper(f, b, a, n)

    cdef double step = (b - a) / n
    if (step == 0): # Handle the case when the limits are the same
        return .0

    cdef double y_sum = (f(a) + f(b)) / 2 # Notice that the coefficients of the endpoints is step/2
    cdef double x = a + step

    for _ in range(n - 1): # Go through all the required x in (a, b)
        # NOTE: We add the complexity of another int to save accuracy
        # If we did while x < b we might get extra points due to floating point errors
        y_sum += f(x)
        x += step

    return y_sum * step



@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.cdivision(True)
@cython.profile(False)
cpdef double cy_trapz_plus(object f_capsule, double a, double b, int n):
    """
    Performs the definite integration of the function f between a and b
    NOTE: Expects a function that is integrable in the range provided!

    Expects a c function!
    """

    # In order to pass a cdef function as a parameter, it is required to wrap it
    # I unwrap the function in the line below and call the cdef function :)
    cdef function_type f = <function_type>PyCapsule_GetPointer(
        f_capsule, PyCapsule_GetName(f_capsule)
    )
    return cy_trapz_plus_helper(f, a, b, n)

## Using NumPy

In [20]:
from typing import Callable


def np_trapz(f: Callable[[float], float], a: float, b: float, n: int) -> float:
    """
    Performs the definite integration of the function f between a and b
    NOTE: Expects a function that is integrable in the range provided!
    """
    assert n > 0, "Cannot integerate function with less than 1 trapezoid!"
    
    if (b < a): # Handle the case when the limits are reversed
        return -np_trapz(f, b, a, n)

    step = (b - a) / n
    if (step == 0): # Handle the case when the limits are the same
        return .0

    # Just makes the function operable on an array (exists just for convenience)
    f_np = np.vectorize(pyfunc=f, otypes=[float])
    xs = np.linspace(a, b, n + 1) # Linspace returns m points (where m is the 3rd arg)
    # We require n + 1 points for n trapezoids :)
    
    return np.trapz(f_np(xs), xs, (b - a) / n)



In [21]:
from typing import Callable, Any


def np_trapz_plus(f: Callable[[Any], Any], a: float, b: float, n: int) -> float:
    """
    Performs the definite integration of the function f between a and b
    NOTE: Expects a function that is integrable in the range provided!

    Expects a function that can operate on a numpy array!
    """
    assert n > 0, "Cannot integerate function with less than 1 trapezoid!"
    
    if (b < a): # Handle the case when the limits are reversed
        return -np_trapz_plus(f, b, a, n)

    step = (b - a) / n
    if (step == 0): # Handle the case when the limits are the same
        return .0

    xs = np.linspace(a, b, n + 1) # Linspace returns m points (where m is the 3rd arg)
    # We require n + 1 points for n trapezoids :)
    return np.trapz(f(xs), xs, (b - a) / n) #type: ignore



## Testing different functions

In [22]:
def compute_accuracy(
        true_value: float,
        cy_plus_value: float,
        np_plus_value: float,
        py_value: float,
        cy_value: float,
        np_value: float,
):
    """Computes the accuracy of the different implementations"""
    print("Delta of the cy_trapz_plus =", cy_plus_value - true_value, "| Percentage error =", (cy_plus_value - true_value) * 100 / true_value, "%")
    print("Delta of the np_trapz_plus =", np_plus_value - true_value, "| Percentage error =", (np_plus_value - true_value) * 100 / true_value, "%")
    print("Delta of the py_trapz =", py_value - true_value, "| Percentage error =", (py_value - true_value) * 100 / true_value, "%")
    print("Delta of the cy_trapz =", cy_value - true_value, "| Percentage error =", (cy_value - true_value) * 100 / true_value, "%")
    print("Delta of the np_trapz =", np_value - true_value, "| Percentage error =", (np_value - true_value) * 100 / true_value, "%")
    


### Pure python functions
The generic python function, can be used with all the trapz methods defined

In [23]:
def pquadratic(x: float) -> float:
    return x * x

def psin(x: float) -> float:
    return math.sin(x)

def pexponential(x: float) -> float:
    return math.exp(x)

def preciprocal(x: float) -> float:
    return 1 / x


### Cython functions
A c function, to be used only with cy_trapz_plus

In [24]:
%%cython --annotate
cimport cython

from libc.math cimport sin as unwrapped_csin
from libc.math cimport exp as unwrapped_cexponential
from cpython.pycapsule cimport PyCapsule_New

@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.cdivision(True)
@cython.profile(False)
cdef double unwrapped_cquadratic(double x):
    return x * x
  
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.cdivision(True)
@cython.profile(False)
cdef double unwrapped_creciprocal(double x):
    return 1 / x

# Wrap the c functions in order to use them in python :)
csin = PyCapsule_New(<void*>unwrapped_csin, 'unwrapped_csin', NULL)
cquadratic = PyCapsule_New(<void*>unwrapped_cquadratic, 'unwrapped_cquadratic', NULL)
cexponential = PyCapsule_New(<void*>unwrapped_cexponential, 'unwrapped_cexponential', NULL)
creciprocal = PyCapsule_New(<void*>unwrapped_creciprocal, 'unwrapped_creciprocal', NULL)


### Numpy functions
A function that can operate on a numpy array

In [25]:
from typing import Any


def nquadratic(x: Any) -> Any:
    return x * x

def nsin(x: Any) -> Any:
    return np.sin(x)

def nexponential(x: Any) -> Any:
    return np.exp(x)

def nreciprocal(x: Any) -> Any:
    return np.reciprocal(x)

### Comparing performance and accuracy of the implementations on different functions

In [26]:
# Checking the quadratic function
QUADRATIC_TRUE_VALUE = 1 / 3

print("cy_trapz_plus: ", end="")
%timeit cy_trapz_plus(cquadratic, 0, 1, 10_000)
print("np_trapz_plus: ", end="")
%timeit np_trapz_plus(nquadratic, 0, 1, 10_000)
print("py_trapz: ", end="")
%timeit py_trapz(pquadratic, 0, 1, 10_000)
print("cy_trapz: ", end="")
%timeit cy_trapz(pquadratic, 0, 1, 10_000)
print("np_trapz: ", end="")
%timeit np_trapz(pquadratic, 0, 1, 10_000)

compute_accuracy(
    QUADRATIC_TRUE_VALUE,
    cy_trapz_plus(cquadratic, 0, 1, 10_000),  # noqa: F821 # type: ignore
    np_trapz_plus(nquadratic, 0, 1, 10_000),
    py_trapz(pquadratic, 0, 1, 10_000),
    cy_trapz(pquadratic, 0, 1, 10_000), # type: ignore  # noqa: F821
    np_trapz(pquadratic, 0, 1, 10_000)
)

cy_trapz_plus: 38.5 μs ± 3.49 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
np_trapz_plus: 97.7 μs ± 4.51 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
py_trapz: 1.5 ms ± 119 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
cy_trapz: 1.49 ms ± 108 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
np_trapz: 2.32 ms ± 230 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Delta of the cy_trapz_plus = 1.6666098501261217e-09 | Percentage error = 4.999829550378365e-07 %
Delta of the np_trapz_plus = 1.6666667490561338e-09 | Percentage error = 5.000000247168401e-07 %
Delta of the py_trapz = 1.6666098501261217e-09 | Percentage error = 4.999829550378365e-07 %
Delta of the cy_trapz = 1.6666098501261217e-09 | Percentage error = 4.999829550378365e-07 %
Delta of the np_trapz = 1.6666667490561338e-09 | Percentage error = 5.000000247168401e-07 %


In [27]:
# Checking the sin function
SIN_TRUE_VALUE = 2

print("cy_trapz_plus: ", end="")
%timeit cy_trapz_plus(csin, 0, math.pi, 10_000)
print("np_trapz_plus: ", end="")
%timeit np_trapz_plus(nsin, 0, math.pi, 10_000)
print("py_trapz: ", end="")
%timeit py_trapz(psin, 0, math.pi, 10_000)
print("cy_trapz: ", end="")
%timeit cy_trapz(psin, 0, math.pi, 10_000)
print("np_trapz: ", end="")
%timeit np_trapz(psin, 0, math.pi, 10_000)

compute_accuracy(
    SIN_TRUE_VALUE,
    cy_trapz_plus(csin, 0, math.pi, 10_000),  # noqa: F821 # type: ignore
    np_trapz_plus(nsin, 0, math.pi, 10_000),
    py_trapz(psin, 0, math.pi, 10_000),
    cy_trapz(psin, 0, math.pi, 10_000), # type: ignore  # noqa: F821
    np_trapz(psin, 0, math.pi, 10_000)
)

cy_trapz_plus: 104 μs ± 12.8 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
np_trapz_plus: 193 μs ± 11.6 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
py_trapz: 3.04 ms ± 293 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
cy_trapz: 3.05 ms ± 356 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
np_trapz: 3.23 ms ± 227 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Delta of the cy_trapz_plus = -1.6449633211124137e-08 | Percentage error = -8.224816605562069e-07 %
Delta of the np_trapz_plus = -1.6449340556334846e-08 | Percentage error = -8.224670278167423e-07 %
Delta of the py_trapz = -1.6449633211124137e-08 | Percentage error = -8.224816605562069e-07 %
Delta of the cy_trapz = -1.6449633211124137e-08 | Percentage error = -8.224816605562069e-07 %
Delta of the np_trapz = -1.6449340556334846e-08 | Percentage error = -8.224670278167423e-07 %


In [28]:
# Checking the exponential function
EXPONENTIAL_TRUE_VALUE = math.exp(1) - 1

print("cy_trapz_plus: ", end="")
%timeit cy_trapz_plus(cexponential, 0, 1, 10_000)
print("np_trapz_plus: ", end="")
%timeit np_trapz_plus(nexponential, 0, 1, 10_000)
print("py_trapz: ", end="")
%timeit py_trapz(pexponential, 0, 1, 10_000)
print("cy_trapz: ", end="")
%timeit cy_trapz(pexponential, 0, 1, 10_000)
print("np_trapz: ", end="")
%timeit np_trapz(pexponential, 0, 1, 10_000)

compute_accuracy(
    EXPONENTIAL_TRUE_VALUE,
    cy_trapz_plus(cexponential, 0, 1, 10_000),  # noqa: F821 # type: ignore
    np_trapz_plus(nexponential, 0, 1, 10_000),
    py_trapz(pexponential, 0, 1, 10_000),
    cy_trapz(pexponential, 0, 1, 10_000), # type: ignore  # noqa: F821
    np_trapz(pexponential, 0, 1, 10_000)
)

cy_trapz_plus: 86.7 μs ± 2.26 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
np_trapz_plus: 177 μs ± 7.17 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
py_trapz: 2.76 ms ± 551 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
cy_trapz: 2.49 ms ± 146 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
np_trapz: 3.37 ms ± 151 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Delta of the cy_trapz_plus = 1.4318228824805601e-09 | Percentage error = 8.33287565966183e-08 %
Delta of the np_trapz_plus = 1.4319017083153085e-09 | Percentage error = 8.333334407659061e-08 %
Delta of the py_trapz = 1.4318228824805601e-09 | Percentage error = 8.33287565966183e-08 %
Delta of the cy_trapz = 1.4318228824805601e-09 | Percentage error = 8.33287565966183e-08 %
Delta of the np_trapz = 1.4319017083153085e-09 | Percentage error = 8.333334407659061e-08 %


In [29]:
# Checking the reciprocal function
RECIPROCAL_TRUE_VALUE = math.log(2)

print("cy_trapz_plus: ", end="")
%timeit cy_trapz_plus(creciprocal, 1, 2, 10_000)
print("np_trapz_plus: ", end="")
%timeit np_trapz_plus(nreciprocal, 1, 2, 10_000)
print("py_trapz: ", end="")
%timeit py_trapz(preciprocal, 1, 2, 10_000)
print("cy_trapz: ", end="")
%timeit cy_trapz(preciprocal, 1, 2, 10_000)
print("np_trapz: ", end="")
%timeit np_trapz(preciprocal, 1, 2, 10_000)

compute_accuracy(
    RECIPROCAL_TRUE_VALUE,
    cy_trapz_plus(creciprocal, 1, 2, 10_000),  # noqa: F821 # type: ignore
    np_trapz_plus(nreciprocal, 1, 2, 10_000),
    py_trapz(preciprocal, 1, 2, 10_000),
    cy_trapz(preciprocal, 1, 2, 10_000), # type: ignore  # noqa: F821
    np_trapz(preciprocal, 1, 2, 10_000)
)

cy_trapz_plus: 38.8 μs ± 1.93 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
np_trapz_plus: 100 μs ± 6.62 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
py_trapz: 1.8 ms ± 47.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
cy_trapz: 1.82 ms ± 41.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
np_trapz: 2.49 ms ± 199 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Delta of the cy_trapz_plus = 6.250213679948047e-10 | Percentage error = 9.017152280557405e-08 %
Delta of the np_trapz_plus = 6.249999406904294e-10 | Percentage error = 9.016843149899787e-08 %
Delta of the py_trapz = 6.250213679948047e-10 | Percentage error = 9.017152280557405e-08 %
Delta of the cy_trapz = 6.250213679948047e-10 | Percentage error = 9.017152280557405e-08 %
Delta of the np_trapz = 6.249999406904294e-10 | Percentage error = 9.016843149899787e-08 %


### Performance test on the quadratic

In [30]:
print("cy_trapz_plus: ", end="")
%timeit cy_trapz_plus(cquadratic, 0, 10, 10_000_000)
print("np_trapz_plus: ", end="")
%timeit np_trapz_plus(nquadratic, 0, 10, 10_000_000)
print("py_trapz: ", end="")
%timeit py_trapz(pquadratic, 0, 10, 10_000_000)
print("cy_trapz: ", end="")
%timeit cy_trapz(pquadratic, 0, 10, 10_000_000)
print("np_trapz: ", end="")
%timeit np_trapz(pquadratic, 0, 10, 10_000_000)


cy_trapz_plus: 37.6 ms ± 2.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
np_trapz_plus: 207 ms ± 7.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
py_trapz: 1.53 s ± 96.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
cy_trapz: 1.47 s ± 107 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
np_trapz: 2.5 s ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
